In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#### ESM-2-2560 and MolFormer embeddings function

In [ ]:
!pip install fair-esm
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 52.0 MB/s eta 0:00:00


In [ ]:
def esm_embeddings_2560(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long,
  #         or you have too many sequences for transformation in a single converting,
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != esm2_alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t12_35M_UR50D' only has 12 layers, and therefore repr_layers parameters is equal to 12
      results = esm2(batch_tokens, repr_layers=[36], return_contacts=False)
  token_representations = results["representations"][36].cpu()
  del results, batch_tokens
  torch.cuda.empty_cache()
  gc.collect()
  return token_representations[:,1:-1,:].mean(1)


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from rdkit import Chem

model_smiles = AutoModel.from_pretrained("ibm/MoLFormer-XL-both-10pct", deterministic_eval=True, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("ibm/MoLFormer-XL-both-10pct", trust_remote_code=True)

def MolFormer_embedding(model_smiles, tokenizer, SMILES_list):
    inputs = tokenizer(SMILES_list, padding=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model_smiles(**inputs)
    # NOTICE: if you have several smiles in the list, you will find the average embedding of each token will remain the same
    #           no matter which smiles in side the list, however, the padding will based on the longest smiles,
    #           therefore, the last hidden state representation shape:[len, 768] will change for the same smiles in difference smiles list.
    return outputs.pooler_output # shape is [len_list, 768] ; torch tensor;

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

configuration_molformer.py:   0%|          | 0.00/7.60k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- configuration_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_molformer.py:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

tokenization_molformer.py:   0%|          | 0.00/9.48k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- tokenization_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- tokenization_molformer_fast.py
- tokenization_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


vocab.json:   0%|          | 0.00/41.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/54.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

### select specific part from the test datset for following prediction performance evaluation

####  use the prediction model to predict the results in the test dataset (validate the model performance)

In [2]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560_phylo')

In [3]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

import warnings
from tqdm import tqdm
import os
from pathlib import Path
# Define the device
device = "cuda" if torch.cuda.is_available() else "mps" if  torch.backends.mps.is_available() else "cpu" # torch.has_mps or
print("Using device:", device)
if (device == 'cuda'):
    print(f"Device name: {torch.cuda.get_device_name(device.index)}")
    print(f"Device memory: {torch.cuda.get_device_properties(device.index).total_memory / 1024 ** 3} GB")
elif (device == 'mps'):
    print(f"Device name: <mps>")
else:
    print("NOTE: If you have a GPU, consider using it for training.")
    print("      On a Windows machine with NVidia GPU, check this video: https://www.youtube.com/watch?v=GMSjDTU8Zlc")
    print("      On a Mac machine, run: pip3 install --pre torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/nightly/cpu")
device = torch.device(device)

Using device: cuda
Device name: Tesla T4
Device memory: 14.74810791015625 GB


In [5]:
ESP_test_df_enzy = torch.load('ESP_test_df_enzy.pt')
# Load the saved embeddings_results
ESP_test_df_smiles = torch.load('ESP_test_df_smiles.pt')
y_test = torch.load('ESP_test_df_label.pt')
ESP_test_df_enzy_add = torch.load('ESP_test_df_enzy_>2800_<8000.pt')
ESP_test_df_smiles_add = torch.load('ESP_test_df_smiles_>2800_<8000.pt')
y_test_add = torch.load('ESP_test_df_label_>2800_<8000.pt')

ESP_test_df_enzy = torch.cat([ESP_test_df_enzy, ESP_test_df_enzy_add], dim=0)
ESP_test_df_smiles = torch.cat([ESP_test_df_smiles, ESP_test_df_smiles_add], dim=0)
y_test = torch.cat([y_test, y_test_add], dim=0)
print(ESP_test_df_enzy.shape, ESP_test_df_smiles.shape, y_test.shape)

test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

# Create TensorDataset and DataLoaders
batch_size = 16
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)


torch.Size([13336, 2560]) torch.Size([13336, 768]) torch.Size([13336, 1])


In [6]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(2560)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [7]:
loss_fn = nn.MSELoss().to(device)

In [8]:
def run_validation(model, val_loader,loss_fn, device):
    model.eval()
    loss_sum = 0
    num_batch = len(val_loader)
    total_y_true=[]
    total_y_pred=[]
    total_y_prob=[]
    for ESP_val_df_enzy,ESP_val_df_smiles, y_val in val_loader:

        ESP_val_df_enzy = ESP_val_df_enzy.to(device)
        ESP_val_df_smiles = ESP_val_df_smiles.to(device)
        y_val = y_val.squeeze(1).to(device)

        refined_enzy_embed, refined_smiles_embed = model(ESP_val_df_enzy,ESP_val_df_smiles)
        cos_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
        loss = loss_fn(cos_sim, y_val).detach().cpu().numpy()
        loss_sum = loss_sum + loss # count all the loss in the training process
        y_pred = (cos_sim > 0.5).float().cpu().numpy() # if score > 0.5, assign label 1 otherwise 0, transfer to cpu as numpy
        total_y_true.append(y_val.cpu().numpy())
        total_y_pred.append(y_pred)
        total_y_prob.append(cos_sim.detach().cpu().numpy())

    loss_sum = loss_sum/num_batch # get the overall average loss (Notice: this method is not 100% accurate)

    arrange_y_true = np.concatenate(total_y_true, axis=0)
    arrange_y_pred = np.concatenate(total_y_pred, axis=0)
    arrange_y_prob = np.concatenate(total_y_prob, axis=0)
    tn,fp,fn,tp = confusion_matrix(arrange_y_true, arrange_y_pred).ravel()
    acc = (tp+tn)/(tp+tn+fp+fn)
    specificity = tn/(tn+fp)
    sensitivity = tp/(tp+fn)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    bacc = (sensitivity + specificity)/2
    MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    AUC = roc_auc_score(arrange_y_true, arrange_y_prob)
    f1 = 2*precision*recall/(precision+recall)
    print("loss_sum= ",loss_sum, "ACC= ",acc, "bacc= ",bacc, "precision= ",precision,"specificity= ",specificity, "sensitivity= ",sensitivity, "recall= ",recall, "MCC= ",MCC, "AUC= ",AUC, "f1= ",f1)
    return loss_sum, acc, bacc   # , precision, sensitivity, recall, MCC, AUC, f1


In [12]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [13]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model_esm2_2560_add_>2800_<8000_in_valid2_ACC=0.9357.pt'
# Load the entire model
model_test = torch.load(load_path)
run_validation(model_test,test_loader,loss_fn, device)

loss_sum=  0.06158351360281988 ACC=  0.9356628674265147 bacc=  0.9051909339672698 precision=  0.9084204056545789 specificity=  0.969653767820774 sensitivity=  0.8407281001137656 recall=  0.8407281001137656 MCC=  0.8313574088968375 AUC=  0.9593826968481587 f1=  0.8732644017725258


(0.06158351360281988, 0.9356628674265147, 0.9051909339672698)

In [11]:
## looks great because the loaded model is the correct model we plan to use for evaluation

### model performance in the unseen and lower occurance molecules


#### embedding of smaller dataset separated from test dataset (bypass this block if you have embedded)

In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/analyze_final_model')

In [ ]:
df_0 = pd.read_csv('enzyme_sequence_lower_40.csv', header= 0)
df_1 = pd.read_csv('enzyme_sequence_40_60.csv', header= 0)
df_2 = pd.read_csv('enzyme_sequence_60_80.csv', header= 0)


In [ ]:
import numpy as np
import pandas as pd
import esm
model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()
# generate the peptide embeddings
embeddings_results_enzy = []
embeddings_results_smiles = []
embeddings_results_label = []
for i in range(df_0.shape[0]):
    seq_enzy = df_0['Protein sequence'].iloc[i]
    seq_smiles = df_0['SMILES'].iloc[i]
    if len(seq_enzy) < 5500:
        # print(len(seq_enzy))
        tuple_sequence = tuple(['protein',seq_enzy])
        peptide_sequence_list = []
        peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
        embeddings_results_enzy.append(one_seq_embeddings)
        # the smiles embeddings
        smiles_list = []
        smiles_list.append(Chem.CanonSmiles(seq_smiles)) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = MolFormer_embedding(model_smiles, tokenizer, smiles_list)
        embeddings_results_smiles.append(one_seq_embeddings)
        # record the lable info
        label = torch.tensor(df_0['output'].iloc[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        embeddings_results_label.append(label)
        # print(seq_enzy, seq_smiles)
        # print(i)
embeddings_results_enzy_torch = torch.cat(embeddings_results_enzy, dim=0)
torch.save(embeddings_results_enzy_torch, 'ESP_test_df_enzy_enzyme_sequence_lower_40.pt')

embeddings_results_smiles_torch = torch.cat(embeddings_results_smiles, dim=0)
torch.save(embeddings_results_smiles_torch, 'ESP_test_df_smiles_enzyme_sequence_lower_40.pt')

embeddings_results_label_torch = torch.cat(embeddings_results_label, dim=0)
torch.save(embeddings_results_label_torch, 'ESP_test_df_label_enzyme_sequence_lower_40.pt')


[21:02:07] WARNING: not removing hydrogen atom without neighbors
[21:04:11] WARNING: not removing hydrogen atom without neighbors
[21:07:19] WARNING: not removing hydrogen atom without neighbors
[21:09:49] WARNING: not removing hydrogen atom without neighbors
[21:11:36] WARNING: not removing hydrogen atom without neighbors
[21:12:25] WARNING: not removing hydrogen atom without neighbors
[21:27:55] WARNING: not removing hydrogen atom without neighbors
[21:27:59] WARNING: not removing hydrogen atom without neighbors
[21:45:35] WARNING: not removing hydrogen atom without neighbors
[21:48:16] WARNING: not removing hydrogen atom without neighbors
[21:49:22] WARNING: not removing hydrogen atom without neighbors


In [ ]:
import numpy as np
import pandas as pd
import esm
model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()
# generate the peptide embeddings
embeddings_results_enzy = []
embeddings_results_smiles = []
embeddings_results_label = []
for i in range(df_1.shape[0]):
    seq_enzy = df_1['Protein sequence'].iloc[i]
    seq_smiles = df_1['SMILES'].iloc[i]
    if len(seq_enzy) < 5500:
        # print(len(seq_enzy))
        tuple_sequence = tuple(['protein',seq_enzy])
        peptide_sequence_list = []
        peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
        embeddings_results_enzy.append(one_seq_embeddings)
        # the smiles embeddings
        smiles_list = []
        smiles_list.append(Chem.CanonSmiles(seq_smiles)) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = MolFormer_embedding(model_smiles, tokenizer, smiles_list)
        embeddings_results_smiles.append(one_seq_embeddings)
        # record the lable info
        label = torch.tensor(df_1['output'].iloc[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        embeddings_results_label.append(label)
        print(seq_enzy, seq_smiles)
        print(i)
embeddings_results_enzy_torch = torch.cat(embeddings_results_enzy, dim=0)
torch.save(embeddings_results_enzy_torch, 'ESP_test_df_enzy_enzyme_sequence_40_60.pt')

embeddings_results_smiles_torch = torch.cat(embeddings_results_smiles, dim=0)
torch.save(embeddings_results_smiles_torch, 'ESP_test_df_smiles_enzyme_sequence_40_60.pt')

embeddings_results_label_torch = torch.cat(embeddings_results_label, dim=0)
torch.save(embeddings_results_label_torch, 'ESP_test_df_label_enzyme_sequence_40_60.pt')


MSRYLLPLSALGTVAGAAVLLKDYVTGGACPSKATIPGKTVIVTGANTGIGKQTALELARRGGNIILACRDMEKCEAAAKDIRGETLNHHVNARHLDLASLKSIREFAAKIIEEEERVDILINNAGVMRCPHWTTEDGFEMQFGVNHLGHFLLTNLLLDKLKASAPSRIINLSSLAHVAGHIDFDDLNWQTRKYNTKAAYCQSKLAIVLFTKELSRRLQGSGVTVNALHPGVARTELGRHTGIHGSTFSSTTLGPIFWLLVKSPELAAQPSTYLAVAEELADVSGKYFDGLKQKAPAPEAEDEEVARRLWAESARLVGLEAPSVREQPLPR N=C([O-])c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)([O-])OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)([O-])[O-])[C@@H]3O)[C@@H](O)[C@H]2O)c1
0
MSRYLLPLSALGTVAGAAVLLKDYVTGGACPSKATIPGKTVIVTGANTGIGKQTALELARRGGNIILACRDMEKCEAAAKDIRGETLNHHVNARHLDLASLKSIREFAAKIIEEEERVDILINNAGVMRCPHWTTEDGFEMQFGVNHLGHFLLTNLLLDKLKASAPSRIINLSSLAHVAGHIDFDDLNWQTRKYNTKAAYCQSKLAIVLFTKELSRRLQGSGVTVNALHPGVARTELGRHTGIHGSTFSSTTLGPIFWLLVKSPELAAQPSTYLAVAEELADVSGKYFDGLKQKAPAPEAEDEEVARRLWAESARLVGLEAPSVREQPLPR CC1=C(/C=C/C(C)=C/C=C/C(C)=C/CO)C(C)(C)CCC1
1
MSRYLLPLSALGTVAGAAVLLKDYVTGGACPSKATIPGKTVIVTGANTGIGKQTALELARRGGNIILACRDMEKCEAAAKDIRGETLNHHVNARHLDLASLKSIREFAAKIIEEEERVDILINNAGVMRCPHWTTEDGFEMQFGVNHL

[21:52:13] WARNING: not removing hydrogen atom without neighbors


MKIAIPKERRPGEDRVAISPEVVKKLVGLGFEVIVEQGAGVGASITDDALTAAGATIASTAAQALSQADVVWKVQRPMTAEEGTDEVALIKEGAVLMCHLGALTNRPVVEALTKRKITAYAMELMPRISRAQSMDILSSQSNLAGYRAVIDGAYEFARAFPMMMTAAGTVPPARVLVFGVGVAGLQAIATAKRLGAVVMATDVRAATKEQVESLGGKFITVDDEAMKTAETAGGYAKEMGEEFRKKQAEAVLKELVKTDIAITTALIPGKPAPVLITEEMVTKMKPGSVIIDLAVEAGGNCPLSEPGKIVVKHGVKIVGHTNVPSRVAADASPLFAKNLLNFLTPHVDKDTKTLVMKLEDETVSGTCVTRDGAIVHPALTGQGA [H+]
242
MKIAIPKERRPGEDRVAISPEVVKKLVGLGFEVIVEQGAGVGASITDDALTAAGATIASTAAQALSQADVVWKVQRPMTAEEGTDEVALIKEGAVLMCHLGALTNRPVVEALTKRKITAYAMELMPRISRAQSMDILSSQSNLAGYRAVIDGAYEFARAFPMMMTAAGTVPPARVLVFGVGVAGLQAIATAKRLGAVVMATDVRAATKEQVESLGGKFITVDDEAMKTAETAGGYAKEMGEEFRKKQAEAVLKELVKTDIAITTALIPGKPAPVLITEEMVTKMKPGSVIIDLAVEAGGNCPLSEPGKIVVKHGVKIVGHTNVPSRVAADASPLFAKNLLNFLTPHVDKDTKTLVMKLEDETVSGTCVTRDGAIVHPALTGQGA NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)C=CC1
243
MKIAIPKERRPGEDRVAISPEVVKKLVGLGFEVIVEQGAGVGASITDDALTAAGATIASTAAQALSQADVVWKVQRPMTAEEGTDEVALI

[21:54:46] WARNING: not removing hydrogen atom without neighbors


MAKVLVLYYSMYGHIETMARAVAEGASKVDGAEVVVKRVPETMPPQLFEKAGGKTQTAPVATPQELADYDAIIFGTPTRFGNMSGQMRTFLDQTGGLWASGALYGKLASVFSSTGTGGGQEQTITSTWTTLAHHGMVIVPIGYAAQELFDVSQVRGGTPYGATTIAGGDGSRQPSQEELSIARYQGEYVAGLAVKLNG [H+]
613
MAKVLVLYYSMYGHIETMARAVAEGASKVDGAEVVVKRVPETMPPQLFEKAGGKTQTAPVATPQELADYDAIIFGTPTRFGNMSGQMRTFLDQTGGLWASGALYGKLASVFSSTGTGGGQEQTITSTWTTLAHHGMVIVPIGYAAQELFDVSQVRGGTPYGATTIAGGDGSRQPSQEELSIARYQGEYVAGLAVKLNG N[C@@H](Cc1cnc[nH]1)C(=O)O
614
MAKVLVLYYSMYGHIETMARAVAEGASKVDGAEVVVKRVPETMPPQLFEKAGGKTQTAPVATPQELADYDAIIFGTPTRFGNMSGQMRTFLDQTGGLWASGALYGKLASVFSSTGTGGGQEQTITSTWTTLAHHGMVIVPIGYAAQELFDVSQVRGGTPYGATTIAGGDGSRQPSQEELSIARYQGEYVAGLAVKLNG C[C@H]([NH3+])[C@H](N)CCCCCC(=O)O
615
MAKVLVLYYSMYGHIETMARAVAEGASKVDGAEVVVKRVPETMPPQLFEKAGGKTQTAPVATPQELADYDAIIFGTPTRFGNMSGQMRTFLDQTGGLWASGALYGKLASVFSSTGTGGGQEQTITSTWTTLAHHGMVIVPIGYAAQELFDVSQVRGGTPYGATTIAGGDGSRQPSQEELSIARYQGEYVAGLAVKLNG N[C@@H](CC(=O)c1ccccc1N=CO)C(=O)O
616
MDILNEFSNVFWSTHIWLPPNTTWADIAPGSRPDVVHANYKDLIWPIPFAAVVMLVRYTLERFWISPVGKSLGIRSSRPKKAANVPIL

[22:07:42] WARNING: not removing hydrogen atom without neighbors


MPRVAIIIYTLYGHVAATAEAEKKGIEAAGGSADIYQVEETLSPEVVKALGGAPKPDYPIATQDTLTEYDAFLFGIPTRFGNFPAQWKAFWDRTGGLWAKGALHGKVAGCFVSTGTGGGNEATIMNSLSTLAHHGIIFVPLGYKNVFAELTNMDEVHGGSPWGAGTIAGSDGSRSPSALELQVHEIQGKTFYETVAKF [H+]
2398
MPRVAIIIYTLYGHVAATAEAEKKGIEAAGGSADIYQVEETLSPEVVKALGGAPKPDYPIATQDTLTEYDAFLFGIPTRFGNFPAQWKAFWDRTGGLWAKGALHGKVAGCFVSTGTGGGNEATIMNSLSTLAHHGIIFVPLGYKNVFAELTNMDEVHGGSPWGAGTIAGSDGSRSPSALELQVHEIQGKTFYETVAKF N[C@@H](CO)C(=O)O
2399
MPRVAIIIYTLYGHVAATAEAEKKGIEAAGGSADIYQVEETLSPEVVKALGGAPKPDYPIATQDTLTEYDAFLFGIPTRFGNFPAQWKAFWDRTGGLWAKGALHGKVAGCFVSTGTGGGNEATIMNSLSTLAHHGIIFVPLGYKNVFAELTNMDEVHGGSPWGAGTIAGSDGSRSPSALELQVHEIQGKTFYETVAKF C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C@@H]1CC[C@@H]2O
2400
MPRVAIIIYTLYGHVAATAEAEKKGIEAAGGSADIYQVEETLSPEVVKALGGAPKPDYPIATQDTLTEYDAFLFGIPTRFGNFPAQWKAFWDRTGGLWAKGALHGKVAGCFVSTGTGGGNEATIMNSLSTLAHHGIIFVPLGYKNVFAELTNMDEVHGGSPWGAGTIAGSDGSRSPSALELQVHEIQGKTFYETVAKF N=C(NCCC[C@H](N)C(=O)O)NC(CC(=O)[O-])C(=O)O
2401
MQFNIPTLLTLFRVILIPFFVLVFYLPVTWSPFAAALIFCVAAVTDWFDGFLARR

In [ ]:
import numpy as np
import pandas as pd
import esm
model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()
# generate the peptide embeddings
embeddings_results_enzy = []
embeddings_results_smiles = []
embeddings_results_label = []
for i in range(df_2.shape[0]):
    seq_enzy = df_2['Protein sequence'].iloc[i]
    seq_smiles = df_2['SMILES'].iloc[i]
    if len(seq_enzy) < 5500:
        # print(len(seq_enzy))
        tuple_sequence = tuple(['protein',seq_enzy])
        peptide_sequence_list = []
        peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
        embeddings_results_enzy.append(one_seq_embeddings)
        # the smiles embeddings
        smiles_list = []
        smiles_list.append(Chem.CanonSmiles(seq_smiles)) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = MolFormer_embedding(model_smiles, tokenizer, smiles_list)
        embeddings_results_smiles.append(one_seq_embeddings)
        # record the lable info
        label = torch.tensor(df_2['output'].iloc[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        embeddings_results_label.append(label)
        # print(seq_enzy, seq_smiles)
        # print(i)
embeddings_results_enzy_torch = torch.cat(embeddings_results_enzy, dim=0)
torch.save(embeddings_results_enzy_torch, 'ESP_test_df_enzy_enzyme_sequence_60_80.pt')

embeddings_results_smiles_torch = torch.cat(embeddings_results_smiles, dim=0)
torch.save(embeddings_results_smiles_torch, 'ESP_test_df_smiles_enzyme_sequence_60_80.pt')

embeddings_results_label_torch = torch.cat(embeddings_results_label, dim=0)
torch.save(embeddings_results_label_torch, 'ESP_test_df_label_enzyme_sequence_60_80.pt')


[22:17:16] WARNING: not removing hydrogen atom without neighbors


####  use the prediction model to predict the results in the subsets from test dataset

In [14]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/analyze_final_model')

In [15]:
def run_validation(model, val_loader,loss_fn, device):
    model.eval()
    loss_sum = 0
    num_batch = len(val_loader)
    total_y_true=[]
    total_y_pred=[]
    total_y_prob=[]
    for ESP_val_df_enzy,ESP_val_df_smiles, y_val in val_loader:

        ESP_val_df_enzy = ESP_val_df_enzy.to(device)
        ESP_val_df_smiles = ESP_val_df_smiles.to(device)
        y_val = y_val.squeeze(1).to(device)

        refined_enzy_embed, refined_smiles_embed = model(ESP_val_df_enzy,ESP_val_df_smiles)
        cos_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
        loss = loss_fn(cos_sim, y_val).detach().cpu().numpy()
        loss_sum = loss_sum + loss # count all the loss in the training process
        y_pred = (cos_sim > 0.5).float().cpu().numpy() # if score > 0.5, assign label 1 otherwise 0, transfer to cpu as numpy
        total_y_true.append(y_val.cpu().numpy())
        total_y_pred.append(y_pred)
        total_y_prob.append(cos_sim.detach().cpu().numpy())

    loss_sum = loss_sum/num_batch # get the overall average loss (Notice: this method is not 100% accurate)

    arrange_y_true = np.concatenate(total_y_true, axis=0)
    arrange_y_pred = np.concatenate(total_y_pred, axis=0)
    arrange_y_prob = np.concatenate(total_y_prob, axis=0)
    tn,fp,fn,tp = confusion_matrix(arrange_y_true, arrange_y_pred).ravel()
    acc = (tp+tn)/(tp+tn+fp+fn)
    specificity = tn/(tn+fp)
    sensitivity = tp/(tp+fn)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    bacc = (sensitivity + specificity)/2
    MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    AUC = roc_auc_score(arrange_y_true, arrange_y_prob)
    f1 = 2*precision*recall/(precision+recall)
    print("loss_sum= ",loss_sum, "ACC= ",acc, "bacc= ",bacc, "precision= ",precision,"specificity= ",specificity, "sensitivity= ",sensitivity, "recall= ",recall, "MCC= ",MCC, "AUC= ",AUC, "f1= ",f1)
    return acc,  AUC ,MCC  # , precision, sensitivity, recall, MCC, AUC, f1


In [16]:
import torch
from torch.utils.data import TensorDataset, DataLoader
ESP_test_df_enzy = torch.load('ESP_test_df_enzy_enzyme_sequence_lower_40.pt')
ESP_test_df_smiles = torch.load('ESP_test_df_smiles_enzyme_sequence_lower_40.pt')
y_test = torch.load('ESP_test_df_label_enzyme_sequence_lower_40.pt')
test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)
# Create TensorDataset and DataLoaders
batch_size = 1
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)

run_validation(model_test,test_loader,loss_fn, device)

loss_sum=  0.07846514114515056 ACC=  0.9146341463414634 bacc=  0.8691864554946009 precision=  0.8926630434782609 specificity=  0.9663400085215168 sensitivity=  0.7720329024676851 recall=  0.7720329024676851 MCC=  0.7752006863019694 AUC=  0.9334351626222841 f1=  0.827977315689981


(0.9146341463414634, 0.9334351626222841, 0.7752006863019694)

In [17]:
import torch
from torch.utils.data import TensorDataset, DataLoader
ESP_test_df_enzy = torch.load('ESP_test_df_enzy_enzyme_sequence_40_60.pt')
ESP_test_df_smiles = torch.load('ESP_test_df_smiles_enzyme_sequence_40_60.pt')
y_test = torch.load('ESP_test_df_label_enzyme_sequence_40_60.pt')
test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)
# Create TensorDataset and DataLoaders
batch_size = 16
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)

run_validation(model_test,test_loader,loss_fn, device)

loss_sum=  0.04900776309248827 ACC=  0.9549483013293943 bacc=  0.9369268318447292 precision=  0.9314285714285714 specificity=  0.9757820383451059 sensitivity=  0.8980716253443526 recall=  0.8980716253443526 MCC=  0.8841541150349774 AUC=  0.9730918486766573 f1=  0.9144460028050491


(0.9549483013293943, 0.9730918486766573, 0.8841541150349774)

In [18]:
import torch
from torch.utils.data import TensorDataset, DataLoader
ESP_test_df_enzy = torch.load('ESP_test_df_enzy_enzyme_sequence_60_80.pt')
ESP_test_df_smiles = torch.load('ESP_test_df_smiles_enzyme_sequence_60_80.pt')
y_test = torch.load('ESP_test_df_label_enzyme_sequence_60_80.pt')
test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)
# Create TensorDataset and DataLoaders
batch_size = 16
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)

run_validation(model_test,test_loader,loss_fn, device)

loss_sum=  0.0397298105705816 ACC=  0.9630973986690865 bacc=  0.9523220667559509 precision=  0.9349775784753364 specificity=  0.9759136212624585 sensitivity=  0.9287305122494433 recall=  0.9287305122494433 MCC=  0.9065528357868837 AUC=  0.9863641240408734 f1=  0.9318435754189945


(0.9630973986690865, 0.9863641240408734, 0.9065528357868837)